In [1]:
from pykrx import stock
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import platform
import datetime
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus'] = False
stock_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
stock_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

kospi_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="ALL")})
kospi_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))

kosdaq_list = pd.DataFrame({'종목코드':stock.get_market_ticker_list(market="KOSDAQ")})
kosdaq_list['종목명'] = stock_list['종목코드'].map(lambda x: stock.get_market_ticker_name(x))


In [2]:
today = datetime.datetime.today()
month_ago = str(today.year) + str(today.month-1) + str(today.day)
month_ago2 = str(today.year) + str(today.month-2) + str(today.day)
today = str(today.year) + str(today.month) + str(today.day)
month_ago

'2021109'

In [3]:
shrink_kospi = stock.get_market_cap_by_ticker(month_ago, market='KOSPI').sort_values('시가총액', ascending=False)
shrink_kospi = shrink_kospi[0:50]
shrink_kosdaq = stock.get_market_cap_by_ticker(month_ago, market='KOSDAQ').sort_values('시가총액', ascending=False)
shrink_kosdaq = shrink_kosdaq[0:50]
name1 = []
name2 = []
for ticker in shrink_kospi.index:
    name1.append(kospi_list[kospi_list['종목코드'] == ticker]['종목명'].values[0])
for ticker in shrink_kosdaq.index:
    name2.append(kosdaq_list[kosdaq_list['종목코드'] == ticker]['종목명'].values[0])
total_name = name1 + name2
len(name1), len(name2), len(total_name)

(50, 50, 100)

In [4]:
def get_name(name):
    name = name
    fromdate = month_ago2
    todate = today

    ticker = stock_list.loc[stock_list['종목명']==name, '종목코드']
    df = stock.get_market_ohlcv_by_date(fromdate=fromdate, todate=todate, ticker=ticker)
    df = df.rename(columns={'시가':'Open', '고가':'High', '저가':'Low', '종가':'Close', '거래량':'Volume'})
    df['ma20'] = df['Close'].rolling(window=20).mean() # 20일 이동표준편차
    df['stddev'] = df['Close'].rolling(window=20).std() # 20일 이동표준편차
    df['upper'] = df['ma20'] + 2*df['stddev'] # 상단밴드
    df['lower'] = df['ma20'] - 2*df['stddev'] # 하단밴드
    df = df[19:] # 20일 이동평균을 구했기 때문에 20번째 행부터 값이 들어가 있음
    df.insert(0, column='volatility', value=0)

    df['volatility'] = (df['upper'] - df['lower'])

    return df[::-1]


In [13]:
select = []
for name in total_name:
    cnt_moment = 0
    mnt = get_name(name)['Close']
    if (mnt[1] > mnt[3]) & (mnt[2] > mnt[4]) & (mnt[3] > mnt[5]):
        select.append(name)

select  
        

['현대차', '기아', '한국전력', 'KT&G', '하이브', '두산중공업', '동양', 'F&F', '아프리카TV']

In [14]:
select2 = []
for name in select:
    cnt_moment = 0
    vol = get_name(name)['volatility']
    if (vol[1] > vol[3]) & (vol[2] > vol[4]) & (vol[3] > vol[5]):
        select2.append(name)

select2  

['KT&G', '하이브', '아프리카TV']

In [ ]:
## back testing
